# MTA Turnstile Data Exploratory Data Analysis

This notebook accesses the collected data from SQL database and does data cleaning and preparation for analysis

In [1]:
# Importing packages
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sqlalchemy import create_engine

In [2]:
# Accessing data from database
engine = create_engine('sqlite:///data/mta.db')
df = pd.read_sql('SELECT * FROM mta_data;',engine)

In [3]:
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/21/2020,00:00:00,REGULAR,0007411940,0002515962 ...
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/21/2020,04:00:00,REGULAR,0007411942,0002515966 ...
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/21/2020,08:00:00,REGULAR,0007411945,0002515979 ...
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/21/2020,12:00:00,REGULAR,0007411969,0002516000 ...
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/21/2020,16:00:00,REGULAR,0007412028,0002516024 ...


In [4]:
print(df.dtypes)
print(df.info)

C/A                                                                     object
UNIT                                                                    object
SCP                                                                     object
STATION                                                                 object
LINENAME                                                                object
DIVISION                                                                object
DATE                                                                    object
TIME                                                                    object
DESC                                                                    object
ENTRIES                                                                 object
EXITS                                                                   object
dtype: object
<bound method DataFrame.info of             C/A  UNIT       SCP        STATION LINENAME DIVISION        DATE  \
0    

### Data Cleaning

#### Manage Column
   - Standardize column names
   - convert column datatype
   - drop unnecessary rows and columns

In [5]:
def data_formatting(df):
    # Rename columns to remove whitespace
    df.columns = [column.strip() for column in df.columns]
    
    # Create DATETIME column from 'DATE' and 'TIME' columns
    df['DATETIME'] = pd.to_datetime(df['DATE'] + " " + df['TIME'],
                                   format="%m/%d/%Y %H:%M:%S")
    
    # Create DAY_OF_WEEK column from DATETIME
    df['DAY_OF_WEEK'] = pd.to_datetime(df['DATETIME']).dt.dayofweek
    
    # Change dtypes
    df['ENTRIES'] = df.ENTRIES.astype("int")
    df['EXITS'] = df.EXITS.astype("int")
    
    # Remove non-REGULAR values from 'DESC'
    df = df.drop(df.loc[df.DESC != 'REGULAR'].index)
    
    # Remove rows for PATH, RIT, and SRT since only checking on subway data
    df.drop(df.loc[df['DIVISION'].isin(['PTH','RIT','SRT'])].index, inplace=True)
    
    # Drop unnecessary columns
    df = df.drop(['LINENAME', 'DIVISION', 'DESC', 'DATE', 'TIME'], axis=1)
    
    return df

In [6]:
df2 = data_formatting(df)
df2.head(20)

,C/A,UNIT,SCP,STATION,ENTRIES,EXITS,DATETIME,DAY_OF_WEEK
0,A002,R051,02-00-00,59 ST,7411940,2515962,2020-03-21 00:00:00,5
1,A002,R051,02-00-00,59 ST,7411942,2515966,2020-03-21 04:00:00,5
2,A002,R051,02-00-00,59 ST,7411945,2515979,2020-03-21 08:00:00,5
3,A002,R051,02-00-00,59 ST,7411969,2516000,2020-03-21 12:00:00,5
4,A002,R051,02-00-00,59 ST,7412028,2516024,2020-03-21 16:00:00,5
5,A002,R051,02-00-00,59 ST,7412053,2516040,2020-03-21 20:00:00,5
6,A002,R051,02-00-00,59 ST,7412067,2516049,2020-03-22 00:00:00,6
7,A002,R051,02-00-00,59 ST,7412068,2516049,2020-03-22 04:00:00,6
8,A002,R051,02-00-00,59 ST,7412071,2516059,2020-03-22 08:00:00,6
10,A002,R051,02-00-00,59 ST,7412115,2516088,2020-03-22 16:00:00,6


#### Manage Missing Values

   - Detecting missing values
   - Handle missing values

In [7]:
# Drop NaN created from previous function
print("before dropna: ", df2.shape)
df2.dropna(how='any', inplace=True)
print("after dropna: ", df2.shape)

before dropna:  (10035462, 8)
after dropna:  (10035462, 8)


#### Manage Duplicate Data

   - Detecting duplicates
   - Handle duplicates

In [8]:
# Checking for duplicates for a given turnstile for a given datetime
(df2
.groupby(['C/A','UNIT','SCP','STATION','DATETIME'])
.ENTRIES.count()
.reset_index()
.sort_values('ENTRIES', ascending=False)).head()

,C/A,UNIT,SCP,STATION,DATETIME,ENTRIES
0,A002,R051,02-00-00,59 ST,2020-03-21 00:00:00,1
6690311,R155,R116,01-00-09,50 ST,2020-04-11 00:00:00,1
6690304,R155,R116,01-00-09,50 ST,2020-04-09 20:00:00,1
6690305,R155,R116,01-00-09,50 ST,2020-04-10 00:00:00,1
6690306,R155,R116,01-00-09,50 ST,2020-04-10 04:00:00,1


In [9]:
df2.sort_values(['C/A', 'UNIT', 'SCP', 'STATION', 'DATETIME'],
                      inplace=True, ascending=True)
df2.head(10)

,C/A,UNIT,SCP,STATION,ENTRIES,EXITS,DATETIME,DAY_OF_WEEK
0,A002,R051,02-00-00,59 ST,7411940,2515962,2020-03-21 00:00:00,5
1,A002,R051,02-00-00,59 ST,7411942,2515966,2020-03-21 04:00:00,5
2,A002,R051,02-00-00,59 ST,7411945,2515979,2020-03-21 08:00:00,5
3,A002,R051,02-00-00,59 ST,7411969,2516000,2020-03-21 12:00:00,5
4,A002,R051,02-00-00,59 ST,7412028,2516024,2020-03-21 16:00:00,5
5,A002,R051,02-00-00,59 ST,7412053,2516040,2020-03-21 20:00:00,5
6,A002,R051,02-00-00,59 ST,7412067,2516049,2020-03-22 00:00:00,6
7,A002,R051,02-00-00,59 ST,7412068,2516049,2020-03-22 04:00:00,6
8,A002,R051,02-00-00,59 ST,7412071,2516059,2020-03-22 08:00:00,6
10,A002,R051,02-00-00,59 ST,7412115,2516088,2020-03-22 16:00:00,6


### Data Preparation

#### Make sense of entry and exit counts
   - counter resets
   - counter counting backwards

In [10]:
# Create previous datetime, entries and exits columns
df2[['PREV_DATETIME', "PREV_ENTRIES", "PREV_EXITS"]] = (df2
                                                .groupby(['C/A', 'UNIT', 'SCP', 'STATION'])\
                                                       ['DATETIME','ENTRIES','EXITS']
                                                .apply(lambda grp: grp.shift(1)))

<ipython-input-10-70e22028cda5>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df2[['PREV_DATETIME', "PREV_ENTRIES", "PREV_EXITS"]] = (df2


In [11]:
# Drop the rows for the earliest times in the df
df2.dropna(subset=["PREV_DATETIME"], axis=0, inplace=True)
df2.head()

,C/A,UNIT,SCP,STATION,ENTRIES,EXITS,DATETIME,DAY_OF_WEEK,PREV_DATETIME,PREV_ENTRIES,PREV_EXITS
1,A002,R051,02-00-00,59 ST,7411942,2515966,2020-03-21 04:00:00,5,2020-03-21 00:00:00,7411940.0,2515962.0
2,A002,R051,02-00-00,59 ST,7411945,2515979,2020-03-21 08:00:00,5,2020-03-21 04:00:00,7411942.0,2515966.0
3,A002,R051,02-00-00,59 ST,7411969,2516000,2020-03-21 12:00:00,5,2020-03-21 08:00:00,7411945.0,2515979.0
4,A002,R051,02-00-00,59 ST,7412028,2516024,2020-03-21 16:00:00,5,2020-03-21 12:00:00,7411969.0,2516000.0
5,A002,R051,02-00-00,59 ST,7412053,2516040,2020-03-21 20:00:00,5,2020-03-21 16:00:00,7412028.0,2516024.0


In [12]:
# Functions for entry, exit counts, and traffic counts
def get_entry_counts(row, max_counter):
    counter = abs(row['ENTRIES'] - row['PREV_ENTRIES'])
    # Set anomaly values due to reset of counters to the uniform NaN values
    if counter > max_counter:
        counter = np.nan
    return counter

def get_exit_counts(row, max_counter):
    counter = abs(row['EXITS'] - row['PREV_EXITS'])
    # Set anomaly values due to reset of counters to the uniform NaN values
    if counter > max_counter:
        counter = np.nan
    return counter

def get_counts(df):
    # Set max_counter = 14400, assuming only 1/person/sec/turnstile at a time in a 4HR interval
    # Entry count
    df['entry_count'] = df.apply(get_entry_counts, axis=1, max_counter=14400)
    
    # Exit count
    df['exit_count'] = df.apply(get_exit_counts, axis=1, max_counter=14400)
    
    return df


In [13]:
df3 = get_counts(df2)
df3.head()

,C/A,UNIT,SCP,STATION,ENTRIES,EXITS,DATETIME,DAY_OF_WEEK,PREV_DATETIME,PREV_ENTRIES,PREV_EXITS,entry_count,exit_count
1,A002,R051,02-00-00,59 ST,7411942,2515966,2020-03-21 04:00:00,5,2020-03-21 00:00:00,7411940.0,2515962.0,2.0,4.0
2,A002,R051,02-00-00,59 ST,7411945,2515979,2020-03-21 08:00:00,5,2020-03-21 04:00:00,7411942.0,2515966.0,3.0,13.0
3,A002,R051,02-00-00,59 ST,7411969,2516000,2020-03-21 12:00:00,5,2020-03-21 08:00:00,7411945.0,2515979.0,24.0,21.0
4,A002,R051,02-00-00,59 ST,7412028,2516024,2020-03-21 16:00:00,5,2020-03-21 12:00:00,7411969.0,2516000.0,59.0,24.0
5,A002,R051,02-00-00,59 ST,7412053,2516040,2020-03-21 20:00:00,5,2020-03-21 16:00:00,7412028.0,2516024.0,25.0,16.0


In [14]:
# Cleaning NaN values in entry_count
# For each NaN values, replace it with the mean of values before and after the NaN value
entry_list = list(df3['entry_count'])
ind = 0
for i in entry_list:
    if np.isnan(i) == 1:
        entry_list[ind] = np.nanmean([entry_list[ind-2],entry_list[ind-1],entry_list[ind+1],entry_list[ind+2]])
    ind += 1
    
df3['ENTRY_DIFF'] = entry_list

In [15]:
df3['ENTRY_DIFF'].isna().sum()

0

In [16]:
# Cleaning NaN values in exit_count
# For each NaN values, replace it with the mean of values before and after the NaN value
exit_list = list(df3['exit_count'])
ind = 0
for i in exit_list:
    if np.isnan(i) == 1:
        exit_list[ind] = np.nanmean([exit_list[ind-2],exit_list[ind-1],exit_list[ind+1],exit_list[ind+2]])
    ind += 1
    
df3['EXIT_DIFF'] = exit_list

In [17]:
df3['EXIT_DIFF'].isna().sum()

0

In [18]:
# Get total traffic count
df3['TOTAL_TRAFFIC'] = df3['ENTRY_DIFF'] + df3['EXIT_DIFF']
df3.head()

,C/A,UNIT,SCP,STATION,ENTRIES,EXITS,DATETIME,DAY_OF_WEEK,PREV_DATETIME,PREV_ENTRIES,PREV_EXITS,entry_count,exit_count,ENTRY_DIFF,EXIT_DIFF,TOTAL_TRAFFIC
1,A002,R051,02-00-00,59 ST,7411942,2515966,2020-03-21 04:00:00,5,2020-03-21 00:00:00,7411940.0,2515962.0,2.0,4.0,2.0,4.0,6.0
2,A002,R051,02-00-00,59 ST,7411945,2515979,2020-03-21 08:00:00,5,2020-03-21 04:00:00,7411942.0,2515966.0,3.0,13.0,3.0,13.0,16.0
3,A002,R051,02-00-00,59 ST,7411969,2516000,2020-03-21 12:00:00,5,2020-03-21 08:00:00,7411945.0,2515979.0,24.0,21.0,24.0,21.0,45.0
4,A002,R051,02-00-00,59 ST,7412028,2516024,2020-03-21 16:00:00,5,2020-03-21 12:00:00,7411969.0,2516000.0,59.0,24.0,59.0,24.0,83.0
5,A002,R051,02-00-00,59 ST,7412053,2516040,2020-03-21 20:00:00,5,2020-03-21 16:00:00,7412028.0,2516024.0,25.0,16.0,25.0,16.0,41.0


#### Unifying Time Intervals

In [19]:
# Check TIME intervals
df3['TIME'] = df3['DATETIME'].apply(lambda x: x.time())
print(df3.TIME.unique())
print(df3.TIME.value_counts())

[datetime.time(4, 0) datetime.time(8, 0) datetime.time(12, 0) ...
 datetime.time(13, 27, 15) datetime.time(1, 12, 15)
 datetime.time(11, 42, 1)]
20:00:00    821505
16:00:00    821351
12:00:00    821214
08:00:00    821027
04:00:00    820885
             ...  
14:36:49         1
05:00:23         1
21:27:19         1
13:29:05         1
17:59:10         1
Name: TIME, Length: 10374, dtype: int64


In [20]:
# Turn all time intervals to 4-hour frequency
df3 = df3.groupby(['C/A', 'UNIT', 'SCP', 'STATION',
                  pd.Grouper(key='DATETIME', freq='4H'),
                  'DAY_OF_WEEK', 'TIME']).sum().reset_index()
# Double checking on time intervals
df3['TIME'] = df3['DATETIME'].apply(lambda x: x.time())
print(df3.TIME.unique())

[datetime.time(4, 0) datetime.time(8, 0) datetime.time(12, 0)
 datetime.time(16, 0) datetime.time(20, 0) datetime.time(0, 0)]


In [21]:
# Syncing Day_of_week column to the new datetime
df3['DAY_OF_WEEK'] = pd.to_datetime(df3['DATETIME']).dt.dayofweek

In [22]:
#Sanity check
df3.groupby(['C/A','UNIT','SCP','STATION','DATETIME']).sum().reset_index()

In [23]:
#Sanity Check
df2.groupby(['C/A','UNIT','SCP','STATION','DATETIME']).sum().reset_index()

#### Final clean-up and save as csv file

In [24]:
# Last Sanity Check before final clean up
(df3
.groupby(['C/A', 'UNIT', 'SCP', 'STATION', 'DATETIME', 'DAY_OF_WEEK'])
.sum()
.reset_index())

,C/A,UNIT,SCP,STATION,DATETIME,DAY_OF_WEEK,ENTRIES,EXITS,PREV_ENTRIES,PREV_EXITS,entry_count,exit_count,ENTRY_DIFF,EXIT_DIFF,TOTAL_TRAFFIC
0,A002,R051,02-00-00,59 ST,2020-03-21 04:00:00,Saturday,7411942,2515966,7.411940e+06,2.515962e+06,2.0,4.0,2.0,4.0,6.0
1,A002,R051,02-00-00,59 ST,2020-03-21 08:00:00,Saturday,7411945,2515979,7.411942e+06,2.515966e+06,3.0,13.0,3.0,13.0,16.0
2,A002,R051,02-00-00,59 ST,2020-03-21 12:00:00,Saturday,7411969,2516000,7.411945e+06,2.515979e+06,24.0,21.0,24.0,21.0,45.0
3,A002,R051,02-00-00,59 ST,2020-03-21 16:00:00,Saturday,7412028,2516024,7.411969e+06,2.516000e+06,59.0,24.0,59.0,24.0,83.0
4,A002,R051,02-00-00,59 ST,2020-03-21 20:00:00,Saturday,7412053,2516040,7.412028e+06,2.516024e+06,25.0,16.0,25.0,16.0,41.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9932569,R730,R431,00-00-04,EASTCHSTER/DYRE,2021-03-19 04:00:00,Friday,1559819708,1728241817,1.559820e+09,1.728242e+09,4.0,0.0,4.0,0.0,4.0
9932570,R730,R431,00-00-04,EASTCHSTER/DYRE,2021-03-19 08:00:00,Friday,1559819657,1728241830,1.559820e+09,1.728242e+09,51.0,13.0,51.0,13.0,64.0
9932571,R730,R431,00-00-04,EASTCHSTER/DYRE,2021-03-19 12:00:00,Friday,1559819639,1728241837,1.559820e+09,1.728242e+09,18.0,7.0,18.0,7.0,25.0
9932572,R730,R431,00-00-04,EASTCHSTER/DYRE,2021-03-19 16:00:00,Friday,1559819597,1728241859,1.559820e+09,1.728242e+09,42.0,22.0,42.0,22.0,64.0


In [25]:
# Drop unnecessary columns
turnstile_df = df3.drop(['C/A', 'UNIT', 'SCP',
                         'ENTRIES', 'EXITS', 'TIME',
                         'PREV_ENTRIES', 'PREV_EXITS', 
                         'entry_count', 'exit_count'], axis=1)
print(turnstile_df.shape)
turnstile_df.head()

(10030770, 6)


,STATION,DATETIME,DAY_OF_WEEK,ENTRY_DIFF,EXIT_DIFF,TOTAL_TRAFFIC
0,59 ST,2020-03-21 04:00:00,Saturday,2.0,4.0,6.0
1,59 ST,2020-03-21 08:00:00,Saturday,3.0,13.0,16.0
2,59 ST,2020-03-21 12:00:00,Saturday,24.0,21.0,45.0
3,59 ST,2020-03-21 16:00:00,Saturday,59.0,24.0,83.0
4,59 ST,2020-03-21 20:00:00,Saturday,25.0,16.0,41.0


In [26]:
turnstile_df.to_csv('./data/cleaned_mta_df.csv')

cleaned dataset saved as csv file for further anaylsis